In [1]:
# [1] Install Required Packages
!pip install datasets
!pip install scikit-learn
!pip install ibm-watson-machine-learning==1.0.312

In [2]:
import os, types, getpass
import time
import pandas as pd
from botocore.client import Config
import ibm_boto3
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model

In [7]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",  # change if not us-south
    "apikey": getpass.getpass("Enter your IBM Cloud API key: ")
}
bucket = 'bucket-hotelreviewh09j8gbiwzne5nv'

Enter your IBM Cloud API key:  ········


In [8]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id only (no &context): ")

In [9]:
#[2] Function to read CSV from Cloud Object Storage
def read_cos_csv(object_key):
    cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=credentials["apikey"],
        ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
        config=Config(signature_version='oauth'),
        endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud'
    )

    body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
    if not hasattr(body, "_iter"): 
        body.iter_ = types.MethodType(lambda self: 0, body)
    return pd.read_csv(body)

In [11]:
# [3] Load Datasets
train_data = read_cos_csv('hotel_review_train.csv')
test_data = read_cos_csv('hotel_review_test.csv')

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (35, 2)
Test data shape: (15, 2)


In [12]:
# [4]Set your project_id (from the URL of your project page)
project_id = 'efaa6757-9b7d-4288-816e-9eb712a68119'  

# Import Model
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

# Choose your model
model_id = ModelTypes.FLAN_UL2

# Parameters using plain keys
parameters = {
    'max_new_tokens': 15
}

# Create the model
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id  
)

In [13]:
print("Test Data Columns:\n", test_data.columns.tolist())

Test Data Columns:
 ['review', 'sentiment']


In [14]:
# [5] Sentiment Prediction Prompt
satisfaction_instruction = (
    "Was the customer satisfied based on the hotel review?\n"
    "comment: The room was dirty and staff were rude.\n"
    "satisfaction: 0\n\n"
    "comment: Excellent stay, clean rooms and friendly staff.\n"
    "satisfaction: 1\n\n"
)

comments = list(test_data['review'])  
results = []

for input_text in comments:
    prompt = satisfaction_instruction + "comment: " + input_text + "\nsatisfaction:"
    result = model.generate_text(prompt=prompt)
    results.append(result)
    time.sleep(8) 

print("\n--- SATISFACTION PREDICTIONS ---")
for comment, prediction in zip(comments, results):
    print("\nREVIEW:", comment)
    print("PREDICTED SATISFACTION:", prediction)


--- SATISFACTION PREDICTIONS ---

REVIEW: After getting bit disappointed by reception staff behaviour on the other hand we got such a lovely hospitality at JW Marriott K3 restaurant.
Here,we would like to say special thanks to K3 restaurant’s staff member Mr.Anuj and his team for making our day by showing top hospitality at the time of dinner on 14Aug and breakfast on busiest morning of 15Aug.  Mr. Anuj was looking  after each and every guest in such an enthusiastic way   we felt so special,he took proper care of everything both days at the busiest time of dinner and breakfast. Food and hospitality at K3 restaurant and staff was brilliant. Thanks to JW Marriott’s Mr. Anuj at K3 restaurant for bringing smile on our faces with his top hospitality.
PREDICTED SATISFACTION: 3

REVIEW: Locality was good staff was good
PREDICTED SATISFACTION: 4

REVIEW:  no comments available for this review
PREDICTED SATISFACTION: 0

REVIEW: The bed was extremely comfortable. The room was spacious and clean

In [15]:
# [6] Business Area Prediction Prompt
business_instruction = (
    "Classify the hotel review into one of these business areas:\n"
    "'Room Service', 'Cleanliness', 'Amenities', 'Staff Behavior', 'Booking Issues', 'Billing and Payment'.\n\n"
    "comment: The receptionist was rude and didn’t help us with check-in.\n"
    "business area: Staff Behavior\n\n"
    "comment: We were charged extra for water bottles in the room.\n"
    "business area: Billing and Payment\n\n"
)

business_results = []
for input_text in comments:
    prompt = business_instruction + "comment: " + input_text + "\nbusiness area:"
    result = model.generate_text(prompt=prompt)
    business_results.append(result)
    time.sleep(8)

In [16]:
# [7] Display Output
for i in range(len(comments)):
    print("\n-----------------------------")
    print("REVIEW:", comments[i])
    print("PREDICTED SATISFACTION:", results[i])
    print("PREDICTED BUSINESS AREA:", business_results[i])


-----------------------------
REVIEW: After getting bit disappointed by reception staff behaviour on the other hand we got such a lovely hospitality at JW Marriott K3 restaurant.
Here,we would like to say special thanks to K3 restaurant’s staff member Mr.Anuj and his team for making our day by showing top hospitality at the time of dinner on 14Aug and breakfast on busiest morning of 15Aug.  Mr. Anuj was looking  after each and every guest in such an enthusiastic way   we felt so special,he took proper care of everything both days at the busiest time of dinner and breakfast. Food and hospitality at K3 restaurant and staff was brilliant. Thanks to JW Marriott’s Mr. Anuj at K3 restaurant for bringing smile on our faces with his top hospitality.
PREDICTED SATISFACTION: 3
PREDICTED BUSINESS AREA: Staff Behavior

-----------------------------
REVIEW: Locality was good staff was good
PREDICTED SATISFACTION: 4
PREDICTED BUSINESS AREA: Staff Behavior

-----------------------------
REVIEW:  no 

In [22]:
# [8] Final Output
output_df = pd.DataFrame({
    "Review": comments,
    "Predicted_Satisfaction": results,
    "Predicted_Business_Area": business_results
})

output_df.to_csv("watsonx_predictions_output.csv", index=False)
print("Output saved to watsonx_predictions_output.csv")

Output saved to watsonx_predictions_output.csv
